
# Sprint 7 — Webinar 7 Teórico  
## Introducción a SQL para Data Analytics (desde cero) — KPIs con un esquema tipo *Bookstore*

<div style="text-align: center">
  <img src="https://raw.githubusercontent.com/ljpiere/tpdata_python/main/Images/books_schema.png" width="550"/>
</div>

> **Duración:** 100 minutos  
> **Modalidad:** Teórico–demostrativa + ejercicios guiados (muy básicos → intermedios)  
> **Herramienta sugerida (navegador):** https://sql-workbench.com/ (DuckDB)  
> **Alternativa dentro de este notebook:** SQLite (para ejecutar consultas y ver resultados aquí mismo)

---

### 🎯 Objetivos de aprendizaje

Al finalizar esta clase, podrás:

1. Explicar qué es SQL y cómo “piensa” una base de datos relacional.
2. Leer un esquema (tablas, llaves primarias/foráneas) y anticipar cómo unir tablas.
3. Escribir consultas con `SELECT`, `WHERE`, `ORDER BY`, `LIMIT`.
4. Calcular métricas con `GROUP BY`, agregaciones (`COUNT`, `SUM`, `AVG`) y `HAVING`.
5. Usar funciones básicas (`CAST`, fechas, `CASE WHEN`) sin perderte con tipos y `NULL`.
6. Aplicar buenas prácticas para escribir SQL legible y “analista-friendly”.

---

### 🧭 Agenda (100 min)

- **0. Setup y reglas del juego (10 min)**
- **1) Entendiendo el esquema relacional (20 min)**
- **2) Llaves primarias, foráneas y relaciones (10 min)**
- **3) Inspeccionar tablas con SQL (10 min)**
- **4) Consultas básicas: `SELECT`, `WHERE`, `ORDER BY` (20 min)**
- **5) Agrupar y agregar datos con `GROUP BY` (25 min)**
- **6) Precisión con tipos y funciones (10–15 min)**
- **7) Buenas prácticas para SQL limpio (10 min)**
- **Cierre + siguientes pasos (5 min)**

---

### 🧠 Importante: “SQL del mundo real”
En tu carrera vas a usar SQL en **muchos motores** (PostgreSQL, MySQL, BigQuery, SQL Server, Snowflake, DuckDB, SQLite…).  
Los **conceptos** son los mismos; cambia el *dialecto* (algunas funciones y detalles).

En esta clase:
- **En navegador** usaremos **DuckDB** (vía sql-workbench.com) porque permite leer CSV desde URL con `read_csv_auto`.
- **En este notebook** usaremos **SQLite** para que puedas ejecutar consultas y ver los resultados sin depender del navegador.



### Conexión a la base de datos

Hoy trabajaremos con un dataset tipo **Bookstore** (autores, libros, clientes, órdenes, items).

Tienes dos opciones:

#### Opción A — SQL en navegador (recomendado en clase)
1. Abre https://sql-workbench.com/  
2. Elige **DuckDB** (modo local en el navegador).
3. **Carga CSV**:
   - Si tienes archivos locales: usa la opción de *import / upload* (drag & drop).
   - Si el dataset está en GitHub como archivo “raw”, puedes leerlo desde SQL (DuckDB) así:

```sql
SELECT *
FROM read_csv_auto('https://raw.githubusercontent.com/tobilg/public-cloud-provider-ip-ranges/main/data/providers/all.csv')
LIMIT 10;
```

> Nota: `read_csv_auto` es específico de DuckDB. En otros motores suele existir un “import” distinto.

#### Opción B — Ejecutar SQL aquí en el notebook (para practicar)
En este notebook vamos a:
- **Generar** CSVs del esquema Bookstore.
- **Cargar** esos CSVs a una base SQLite en memoria.
- Ejecutar consultas con `pandas.read_sql_query`.


In [ ]:

# ============================================
# Setup (Python + SQLite) para correr SQL aquí
# ============================================

import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

random.seed(42)
np.random.seed(42)

con = sqlite3.connect(":memory:")

def run_sql(query: str) -> pd.DataFrame:
    """Ejecuta una consulta SQL sobre SQLite y retorna un DataFrame."""
    return pd.read_sql_query(query, con)

print("✅ Conexión SQLite lista.")



## 🧪 Dataset de clase: esquema tipo *Bookstore*

Vamos a crear 5 tablas:

- `authors` (autores)
- `books` (libros)
- `customers` (clientes)
- `orders` (órdenes)
- `order_items` (detalle de libros por orden)

**Idea de negocio:** una tienda vende libros a clientes en diferentes ciudades y canales (web / tienda / marketplace).

> En un proyecto real, este esquema suele venir dado. Aquí lo generamos para practicar.


In [ ]:

# =========================
# 1) Crear tablas sintéticas
# =========================

# --- Authors ---
countries = ["Colombia", "México", "Argentina", "España", "Estados Unidos", "Chile", "Perú"]
authors = pd.DataFrame({
    "author_id": range(1, 31),
    "author_name": [f"Autor {i:02d}" for i in range(1, 31)],
    "country": np.random.choice(countries, size=30, replace=True)
})

# --- Books ---
genres = ["Ficción", "No ficción", "Tecnología", "Negocios", "Historia", "Infantil", "Ciencia"]
n_books = 120
books = pd.DataFrame({
    "book_id": range(1, n_books + 1),
    "title": [f"Libro {i:03d}" for i in range(1, n_books + 1)],
    "genre": np.random.choice(genres, size=n_books, replace=True),
    "publish_year": np.random.randint(2005, 2026, size=n_books),
    "price_usd": np.round(np.random.uniform(6, 55, size=n_books), 2),
    "author_id": np.random.choice(authors["author_id"], size=n_books, replace=True)
})

# --- Customers ---
cities = ["Bogotá", "Medellín", "Cali", "Barranquilla", "Cartagena", "Bucaramanga"]
segments = ["Nuevo", "Regular", "VIP"]
n_customers = 600

start_signup = datetime(2023, 1, 1)
customers = pd.DataFrame({
    "customer_id": range(1, n_customers + 1),
    "full_name": [f"Cliente {i:04d}" for i in range(1, n_customers + 1)],
    "city": np.random.choice(cities, size=n_customers, replace=True, p=[0.35, 0.18, 0.15, 0.12, 0.12, 0.08]),
    "signup_date": [(start_signup + timedelta(days=int(x))).date().isoformat() for x in np.random.randint(0, 720, size=n_customers)],
    "segment": np.random.choice(segments, size=n_customers, replace=True, p=[0.45, 0.45, 0.10])
})

# --- Orders ---
channels = ["web", "tienda", "marketplace"]
statuses = ["paid", "refunded", "canceled"]

n_orders = 2800
start_order = datetime(2024, 1, 1)

orders = pd.DataFrame({
    "order_id": range(1, n_orders + 1),
    "customer_id": np.random.choice(customers["customer_id"], size=n_orders, replace=True),
    "order_date": [(start_order + timedelta(days=int(x))).date().isoformat() for x in np.random.randint(0, 400, size=n_orders)],
    "channel": np.random.choice(channels, size=n_orders, replace=True, p=[0.55, 0.25, 0.20]),
    "status": np.random.choice(statuses, size=n_orders, replace=True, p=[0.90, 0.03, 0.07])
})

# --- Order Items (1 a 5 libros por orden) ---
rows = []
for oid in orders["order_id"]:
    n_items = int(np.random.randint(1, 6))
    chosen_books = np.random.choice(books["book_id"], size=n_items, replace=False)
    for bid in chosen_books:
        qty = int(np.random.randint(1, 4))
        base_price = float(books.loc[books["book_id"] == bid, "price_usd"].iloc[0])
        unit_price = round(base_price * float(np.random.uniform(0.9, 1.05)), 2)
        rows.append((int(oid), int(bid), qty, unit_price))

order_items = pd.DataFrame(rows, columns=["order_id", "book_id", "quantity", "unit_price_usd"])

# =========================
# 2) Cargar a SQLite
# =========================
authors.to_sql("authors", con, index=False, if_exists="replace")
books.to_sql("books", con, index=False, if_exists="replace")
customers.to_sql("customers", con, index=False, if_exists="replace")
orders.to_sql("orders", con, index=False, if_exists="replace")
order_items.to_sql("order_items", con, index=False, if_exists="replace")

print("✅ Tablas creadas y cargadas en SQLite:")
for t in ["authors", "books", "customers", "orders", "order_items"]:
    n = run_sql(f"SELECT COUNT(*) AS n FROM {t};").iloc[0, 0]
    print(f" - {t}: {n:,} filas")



### Vista rápida de las tablas

Antes de escribir SQL, acostúmbrate a *mirar* datos:

- `LIMIT` para ver pocas filas
- `COUNT(*)` para dimensionar
- En SQLite: `PRAGMA table_info(nombre_tabla)` para ver columnas y tipos (o “inferidos”)


In [ ]:
run_sql('SELECT * FROM books LIMIT 5;')

In [ ]:
run_sql('PRAGMA table_info(books);')


## 🧠 Introducción a SQL (Structured Query Language)

### ¿Qué es SQL?
SQL es el lenguaje estándar para **consultar** (y a veces modificar) datos en **bases relacionales**.

Piensa en una base relacional como un conjunto de **tablas**:
- Filas = registros (clientes, órdenes, libros…)
- Columnas = atributos (ciudad, fecha, precio…)

### ¿Cómo funciona una consulta?
1. El motor lee tu consulta.
2. Decide un plan (cómo buscar/filtrar/unir).
3. Devuelve un resultado tabular.

### ¿Dónde se usa?
- Analítica y BI (dashboards, KPIs)
- Data Engineering (modelado, pipelines)
- Producto (experimentos, funnel, cohortes)
- Operaciones (reportes, auditoría)

### Comandos fundamentales (para analistas)
- `SELECT` (seleccionar columnas)
- `FROM` (tabla/fuente)
- `WHERE` (filtrar)
- `ORDER BY` (ordenar)
- `GROUP BY` + agregaciones (resumir)
- `JOIN` (unir tablas)

### 💡 Tips iniciales
- Escribe palabras clave en **MAYÚSCULA** (legibilidad).
- Indenta y usa alias (`o`, `c`, `b`).
- Empieza simple: `SELECT * FROM tabla LIMIT 10;`
- Si algo “no cuadra”, revisa conteos (`COUNT(*)`) y duplicados (`COUNT(DISTINCT ...)`).



## 1) Entendiendo el esquema relacional (20 min)

### ¿Qué es un esquema?
Es el “mapa” de cómo se organizan los datos: qué tablas existen, qué columnas tienen y cómo se relacionan.

### Nuestro esquema Bookstore (mental model)

- Un **autor** escribe muchos **libros**.
- Un **cliente** puede hacer muchas **órdenes**.
- Una **orden** puede tener muchos **items** (libros).
- Un **libro** puede aparecer en muchos **items** (en diferentes órdenes).

Visual (texto):

```
authors (author_id) 1 ────< books (author_id)
customers (customer_id) 1 ────< orders (customer_id)
orders (order_id) 1 ────< order_items (order_id) >──── 1 books (book_id)
```

> Regla de oro: **una FK apunta a una PK** de otra tabla.


In [ ]:

run_sql(
    "SELECT "
    "(SELECT COUNT(*) FROM authors) AS n_authors, "
    "(SELECT COUNT(*) FROM books) AS n_books, "
    "(SELECT COUNT(*) FROM customers) AS n_customers, "
    "(SELECT COUNT(*) FROM orders) AS n_orders, "
    "(SELECT COUNT(*) FROM order_items) AS n_order_items;"
)



### 🧩 Ejercicio 1 (muy básico)

1) ¿Cuál tabla crees que tiene **más filas**? ¿Por qué?  
2) ¿Cuál columna usarías para conectar `orders` con `customers`?  
3) ¿Cuál columna usarías para conectar `order_items` con `books`?

> Pista: busca palabras como `*_id`.

**Ahora valida con SQL**:
- `PRAGMA table_info(tabla);`
- `SELECT * FROM tabla LIMIT 5;`


In [ ]:
run_sql('SELECT * FROM orders LIMIT 5;')


## 2) Llaves primarias, foráneas y relaciones (10 min)

### Llave primaria (PK)
Columna (o conjunto de columnas) que identifica **únicamente** una fila.
- Ej: `customers.customer_id`, `books.book_id`

### Llave foránea (FK)
Columna que **referencia** a una PK en otra tabla.
- Ej: `orders.customer_id` apunta a `customers.customer_id`

### ¿Por qué importa?
Porque define cómo hacemos `JOIN` sin inventarnos relaciones.

#### Ejemplo: “Traer nombre del cliente en cada orden”
- `orders` tiene `customer_id`
- el nombre está en `customers`

Entonces unimos:

```sql
SELECT o.order_id, o.order_date, c.full_name
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id;
```


In [ ]:

run_sql(
"""SELECT 
  o.order_id,
  o.order_date,
  o.channel,
  o.status,
  c.full_name,
  c.city,
  c.segment
FROM orders o
JOIN customers c
  ON o.customer_id = c.customer_id
LIMIT 10;"""
)



### 🧩 Ejercicio 2

Queremos calcular **cuántas órdenes** ha hecho cada cliente.

1) ¿Qué tabla tiene las órdenes?  
2) ¿Qué columna identifica al cliente en esa tabla?  
3) ¿Cómo agruparías por cliente?

Escribe tu intento y luego compara con la solución.


In [ ]:

run_sql(
"""SELECT
  customer_id,
  COUNT(*) AS n_orders
FROM orders
GROUP BY customer_id
ORDER BY n_orders DESC
LIMIT 10;"""
)



## 3) Inspeccionar tablas con SQL (10 min)

Antes de hacer KPIs, siempre inspecciona:

- **Estructura**: columnas y tipos
- **Valores típicos**: ¿qué categorías existen?
- **Calidad básica**: nulos, duplicados, rangos

### Patrones útiles

```sql
-- ver columnas
PRAGMA table_info(books);

-- valores únicos (categóricas)
SELECT DISTINCT genre FROM books;

-- conteos
SELECT COUNT(*) FROM orders;

-- fechas mín/max
SELECT MIN(order_date), MAX(order_date) FROM orders;
```


In [ ]:
run_sql('SELECT DISTINCT genre FROM books ORDER BY genre;')

In [ ]:

run_sql(
"""SELECT 
  MIN(order_date) AS min_date,
  MAX(order_date) AS max_date,
  COUNT(*) AS n_orders
FROM orders;"""
)



## 4) Consultas básicas: `SELECT`, `WHERE`, `ORDER BY` (20 min)

### `SELECT` y `FROM`
Selecciona columnas de una tabla.

```sql
SELECT title, genre, price_usd
FROM books;
```

### `WHERE` (filtrar)
Filtra filas con condiciones.

```sql
SELECT *
FROM books
WHERE genre = 'Tecnología';
```

Operadores comunes:
- Comparación: `=`, `!=`, `>`, `<`, `>=`, `<=`
- Lógicos: `AND`, `OR`, `NOT`
- Rangos: `BETWEEN a AND b`
- Listas: `IN (...)`
- Texto: `LIKE '%algo%'`
- Nulos: `IS NULL` / `IS NOT NULL`

### `ORDER BY` y `LIMIT`
Ordena y trae solo “lo necesario” (ideal para explorar).


In [ ]:

run_sql(
"""SELECT title, genre, price_usd
FROM books
WHERE genre = 'Tecnología'
ORDER BY price_usd DESC
LIMIT 10;"""
)


In [ ]:

run_sql(
"""SELECT *
FROM orders
WHERE status != 'paid'
ORDER BY order_date DESC
LIMIT 10;"""
)



### 🧩 Ejercicio 3

1) Lista los libros publicados desde 2020 (inclusive) y ordénalos por año descendente y precio descendente.  
2) Muestra solo 15 filas.  
3) Devuelve: `title`, `publish_year`, `price_usd`, `genre`.

> Pista: `WHERE publish_year >= 2020`


In [ ]:

run_sql(
"""SELECT
  title,
  publish_year,
  price_usd,
  genre
FROM books
WHERE publish_year >= 2020
ORDER BY publish_year DESC, price_usd DESC
LIMIT 15;"""
)



## 5) Agrupar y agregar datos con `GROUP BY` (25 min)

Cuando preguntas “¿cuánto?”, “¿cuántos?”, “¿promedio?” casi siempre necesitas:

- `GROUP BY` (la dimensión)
- una agregación (`COUNT`, `SUM`, `AVG`, `MIN`, `MAX`)

### Ejemplo: ventas por canal (solo pagadas)

Primero necesitamos el **valor de cada orden**:
- `order_items` tiene `quantity` y `unit_price_usd`
- valor de item = `quantity * unit_price_usd`
- valor de orden = suma de items

Luego unimos con `orders` para filtrar `status='paid'` y agrupar por canal.


In [ ]:

run_sql(
"""SELECT
  o.channel,
  COUNT(DISTINCT o.order_id) AS n_orders,
  ROUND(SUM(oi.quantity * oi.unit_price_usd), 2) AS revenue_usd,
  ROUND(AVG(oi.quantity * oi.unit_price_usd), 2) AS avg_item_value_usd
FROM orders o
JOIN order_items oi
  ON o.order_id = oi.order_id
WHERE o.status = 'paid'
GROUP BY o.channel
ORDER BY revenue_usd DESC;"""
)



### `HAVING` (filtrar grupos)

`WHERE` filtra *filas antes de agrupar*.  
`HAVING` filtra *grupos después de agrupar*.

Ejemplo: géneros con al menos 300 ventas (unidades).


In [ ]:

run_sql(
"""SELECT
  b.genre,
  SUM(oi.quantity) AS units_sold
FROM order_items oi
JOIN books b  ON oi.book_id = b.book_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'paid'
GROUP BY b.genre
HAVING SUM(oi.quantity) >= 300
ORDER BY units_sold DESC;"""
)



### 🧩 Ejercicio 4 (KPIs clásicos)

**Parte A — Top 10 libros por ingresos (solo pagadas)**  
Devuelve: `title`, `genre`, `revenue_usd` ordenado por ingresos desc.

**Parte B — Ticket promedio por ciudad (solo pagadas)**  
Ticket = valor total por orden.  
Devuelve: `city`, `avg_ticket_usd`, `n_orders` (solo si `n_orders >= 80`).

> Pista para B: calcula primero el total por `order_id` y luego agrupa por `city`.


In [ ]:

run_sql(
"""SELECT
  b.title,
  b.genre,
  ROUND(SUM(oi.quantity * oi.unit_price_usd), 2) AS revenue_usd
FROM order_items oi
JOIN books b  ON oi.book_id = b.book_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.status = 'paid'
GROUP BY b.title, b.genre
ORDER BY revenue_usd DESC
LIMIT 10;"""
)


In [ ]:

run_sql(
"""WITH order_totals AS (
  SELECT
    o.order_id,
    o.customer_id,
    ROUND(SUM(oi.quantity * oi.unit_price_usd), 2) AS order_total_usd
  FROM orders o
  JOIN order_items oi ON o.order_id = oi.order_id
  WHERE o.status = 'paid'
  GROUP BY o.order_id, o.customer_id
)
SELECT
  c.city,
  ROUND(AVG(ot.order_total_usd), 2) AS avg_ticket_usd,
  COUNT(*) AS n_orders
FROM order_totals ot
JOIN customers c ON ot.customer_id = c.customer_id
GROUP BY c.city
HAVING COUNT(*) >= 80
ORDER BY avg_ticket_usd DESC;"""
)



## 6) Precisión con tipos y funciones (10–15 min)

### Tipos y `CAST`
En proyectos reales, los tipos importan (texto vs número vs fecha).

En SQLite, `order_date` está como texto ISO (`YYYY-MM-DD`).  
Podemos transformarlo con funciones de fecha.

### Fechas: extraer mes / año
En SQLite podemos usar `strftime`.

### `CASE WHEN` (segmentación)
Sirve para crear categorías nuevas (por ejemplo, tamaño de ticket).


In [ ]:

run_sql(
"""SELECT
  strftime('%Y-%m', order_date) AS year_month,
  COUNT(*) AS n_orders
FROM orders
WHERE status = 'paid'
GROUP BY year_month
ORDER BY year_month;"""
)


In [ ]:

run_sql(
"""WITH order_totals AS (
  SELECT
    o.order_id,
    ROUND(SUM(oi.quantity * oi.unit_price_usd), 2) AS order_total_usd
  FROM orders o
  JOIN order_items oi ON o.order_id = oi.order_id
  WHERE o.status = 'paid'
  GROUP BY o.order_id
)
SELECT
  CASE
    WHEN order_total_usd < 20 THEN 'bajo'
    WHEN order_total_usd < 50 THEN 'medio'
    ELSE 'alto'
  END AS ticket_bucket,
  COUNT(*) AS n_orders,
  ROUND(AVG(order_total_usd), 2) AS avg_ticket_usd
FROM order_totals
GROUP BY ticket_bucket
ORDER BY avg_ticket_usd;"""
)



## 7) Buenas prácticas para SQL limpio (10 min)

- Usa alias claros (`orders o`, `customers c`, `order_items oi`, `books b`).
- Evita `SELECT *` en consultas finales (úsalo solo para explorar).
- Usa CTEs (`WITH`) para pasos intermedios.
- Nombres consistentes (`snake_case`, IDs con `_id`).
- Valida joins con checks rápidos (`COUNT(*)`, `COUNT(DISTINCT ...)`).

#### Ejemplo: chequeo de duplicados por join
Si unes `orders` (1 fila por orden) con `order_items` (varias filas por orden), el resultado tendrá más filas.


In [ ]:

run_sql(
"""SELECT
  COUNT(*) AS rows_after_join,
  COUNT(DISTINCT o.order_id) AS distinct_orders
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id;"""
)



### Bonus opcional (si hay tiempo): funciones de ventana

Ejemplo: top 3 clientes por ingresos (solo pagadas) usando `ROW_NUMBER()`.


In [ ]:

run_sql(
"""WITH customer_revenue AS (
  SELECT
    o.customer_id,
    ROUND(SUM(oi.quantity * oi.unit_price_usd), 2) AS revenue_usd
  FROM orders o
  JOIN order_items oi ON o.order_id = oi.order_id
  WHERE o.status = 'paid'
  GROUP BY o.customer_id
),
ranked AS (
  SELECT
    customer_id,
    revenue_usd,
    ROW_NUMBER() OVER (ORDER BY revenue_usd DESC) AS rn
  FROM customer_revenue
)
SELECT
  r.customer_id,
  c.full_name,
  c.city,
  r.revenue_usd
FROM ranked r
JOIN customers c ON r.customer_id = c.customer_id
WHERE r.rn <= 3
ORDER BY r.revenue_usd DESC;"""
)



## Cierre

### ✅ Lo que ya puedes hacer
- Explorar tablas y entender un esquema relacional.
- Filtrar, ordenar y limitar resultados.
- Calcular KPIs con `GROUP BY` y agregaciones.
- Usar `JOIN` para conectar información.
- Trabajar con fechas y `CASE WHEN` para segmentar.

### 🧩 Mini–reto (tarea)
Construye un “dashboard” (solo consultas) con:
1) Ingresos totales (pagadas)  
2) Ingresos por mes (`YYYY-MM`)  
3) Top 5 géneros por ingresos  
4) Top 10 libros por unidades vendidas  
5) % de órdenes canceladas y reembolsadas

---

## Siguientes pasos (para el proyecto)

Flujo típico:
1. Entender el negocio (preguntas/KPIs).
2. Explorar el modelo de datos.
3. Construir métricas base.
4. Profundizar (segmentación, tendencias).
5. Documentar (queries limpias + supuestos).

Si usas sql-workbench.com (DuckDB) y quieres leer un CSV desde URL:

```sql
CREATE VIEW provider_ips AS
SELECT * 
FROM read_csv_auto('https://raw.githubusercontent.com/tobilg/public-cloud-provider-ip-ranges/main/data/providers/all.csv');
```



### (Opcional) Exportar CSVs para cargarlos en SQL Workbench

Si quieres practicar 100% en el navegador, puedes descargar los CSVs generados por este notebook y subirlos a sql-workbench.com.


In [ ]:

import os
out_dir = "bookstore_csv"
os.makedirs(out_dir, exist_ok=True)

authors.to_csv(f"{out_dir}/authors.csv", index=False)
books.to_csv(f"{out_dir}/books.csv", index=False)
customers.to_csv(f"{out_dir}/customers.csv", index=False)
orders.to_csv(f"{out_dir}/orders.csv", index=False)
order_items.to_csv(f"{out_dir}/order_items.csv", index=False)

out_dir
